In [18]:

import pandas as pd
import numpy as np
import math as m
import os
import csv
import random as rn
from random import randint
from scipy.stats import beta


papers_number = 500
readers_number = 250
authors_number = 25

papers = np.arange(papers_number)
readers = np.arange(readers_number)
authors = np.arange(authors_number)

# Seed folder path

dataset_name = "seed_2/p_1_beta_small"
dataset_folder_path = f"../data/{dataset_name}/"
info_file_path = f"{dataset_folder_path}info.csv"
ratings_file_path = f"{dataset_folder_path}ratings.csv"
authors_file_path = f"{dataset_folder_path}authors.csv"

os.makedirs(dataset_folder_path, exist_ok=True)

print("DATASET NAME: ", dataset_name)
print("DATASET FOLDER PATH: ", dataset_folder_path)
print("INFO FILE PATH: ", info_file_path)
print("RATINGS FILE PATH: ", ratings_file_path)
print("AUTHORS FILE PATH: ", authors_file_path)

# Papers distribution generation with beta distribution

print("---------- PAPER DISTRIBUTIONS GENERATION STARTED ----------")

# CASE 1: a == b == 1, 5% of papers
beta_distributions_frequencies = [(m.floor((5*papers_number)/100), (1, 1))]
# CASE 2: a == b > 1, 30% of papers
a = randint(2, 10)
b = a
beta_distributions_frequencies.append((m.floor((30*papers_number)/100), (a, b)))
# CASE 3: 0 < (a ^ b) < 1, 30% of papers
a = rn.uniform(0.001, 1)
b = rn.uniform(0.001, 1)
beta_distributions_frequencies.append((m.floor((20*papers_number)/100), (a, b)))
# CASE 4: (a V b) == 1, (a > b V b > a), 20% of papers
a = 1
b = randint(1, 10)
if rn.randint(0,1) > 0.5:
    a, b = b, a
beta_distributions_frequencies.append((m.floor((30*papers_number)/100), (a, b)))
# CASE 5: (a ^ b) > 1, (a > b V b > a), 15% of papers
a = randint(2, 10)
b = randint(2 + a, 10 + a)
if rn.randint(0,1) > 0.5:
    a, b = b, a
beta_distributions_frequencies.append((m.floor((15*papers_number)/100), (a, b)))

papers_set = set(papers)
paper_distributions = [None] * papers_number

generated_papers_distributions = 0
for (papers_amount, (a, b)) in beta_distributions_frequencies:
    current_paper_set = rn.sample(papers_set, papers_amount)
    for paper in current_paper_set:
        percentage = 100*generated_papers_distributions/papers_number
        if percentage % 10 == 0:
            print(f"{int(generated_papers_distributions)}/{papers_number} ({int(percentage)}/100%)")
        paper_distributions[paper] = beta(a=a, b=b)
        generated_papers_distributions = generated_papers_distributions + 1
        papers_set.remove(paper)
print(f"{papers_number}/{papers_number} (100/100%)")

print("---------- PAPER DISTRIBUTIONS GENERATION COMPLETED ----------")


DATASET NAME:  seed_2/p_1_beta_small
DATASET FOLDER PATH:  ../data/seed_2/p_1_beta_small/
INFO FILE PATH:  ../data/seed_2/p_1_beta_small/info.csv
RATINGS FILE PATH:  ../data/seed_2/p_1_beta_small/ratings.csv
AUTHORS FILE PATH:  ../data/seed_2/p_1_beta_small/authors.csv
---------- PAPER DISTRIBUTIONS GENERATION STARTED ----------
0/500 (0/100%)
50/500 (10/100%)
100/500 (20/100%)
150/500 (30/100%)
200/500 (40/100%)
250/500 (50/100%)
300/500 (60/100%)
350/500 (70/100%)
400/500 (80/100%)
450/500 (90/100%)
500/500 (100/100%)
---------- PAPER DISTRIBUTIONS GENERATION COMPLETED ----------


In [19]:

# Ratings file generation

# N sets of readers, each one has X% of the total

readers_percent = 20
reader_sets_number = m.floor(100 / readers_percent)
readers_amount = m.floor((readers_number*readers_percent)/100)

readers_sets = []

# Readers rate papers with a certain frequence

paper_frequencies = [2, 4, 8, 30, 90]

print("---------- READERS SETS GENERATION STARTED ----------")

ratings_number = sum(paper_frequencies) * readers_amount
for x in range(0, reader_sets_number):
    current_readers_set = np.random.choice(readers, readers_amount, False) 
    readers = np.setdiff1d(readers, current_readers_set)
    readers_sets.append(current_readers_set)
    print(f"SET {x}: ", current_readers_set)

print("---------- READERS SETS GENERATION COMPLETED ----------")

print("---------- RATINGS GENERATION STARTED ----------")

generated_ratings = 0
rated_papers = []
with open(ratings_file_path, mode='w', newline='') as ratings_file:
    ratings_writer = csv.writer(ratings_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    ratings_writer.writerow(['Timestamp', 'Reader', 'Paper', 'Score'])
    for current_set in range(0, reader_sets_number):
        frequency = paper_frequencies[current_set]
        readers_set = readers_sets[current_set]
        for reader in readers_set:
            sample = np.random.choice(papers, frequency, False)     
            for paper in sample:    
                paper_distribution = paper_distributions[paper]
                percentage = 100*generated_ratings/ratings_number
                if percentage % 10 == 0:
                    print(f"{int(generated_ratings)}/{ratings_number} ({int(percentage)}/100%)")
                generated_rating = round(paper_distribution.rvs(1)[0], 2)
                if generated_rating == 0:
                    generated_rating = 0.01
                ratings_writer.writerow([
                    generated_ratings, 
                    reader, 
                    paper, 
                    generated_rating
                ])
                rated_papers.append(paper)
                generated_ratings+=1
    
    # Filling gaps
    unrated_papers = set(papers) - set(rated_papers)    
    for paper in unrated_papers:
        for reader in np.random.choice(readers, 5, False):
            paper_distribution = paper_distributions[paper]
            generated_rating = round(paper_distribution.rvs(1)[0], 2)
            if generated_rating == 0:
                generated_rating = 0.01
                ratings_writer.writerow([
                    generated_ratings, 
                    reader, 
                    paper,
                    generated_rating
                ])
                generated_ratings+=1
        
    print(f"{ratings_number}/{ratings_number} (100/100%)")
    
ratings_file.close()

print("---------- RATINGS GENERATION ENDED ----------")


---------- READERS SETS GENERATION STARTED ----------
SET 0:  [154  28 114 233 143  52  76 203   7 219 134 200 133  23 129  73 102  11
 242  26 194 158 139 210 204  15 183  27  45 161 121 145 107 116  91 186
 191 179  81 241   3 239 144 135 164 208 123 237 220  75]
SET 1:  [180 211 115 190  99  51 225 120 223 185  97 105 167  13  64  49 187 181
 235  96 172 177 193  94 157 156  68 173 216 151  62 136 130 111  58  29
 249  42 207  92 228  18 128  35 101  72 152  53   9  61]
SET 2:  [230  10 168 103 150 131  14 160  57  46  77 108   8  12 112  88  25 247
 234  74 246 196  30 248 215 132  83  98  55 117  78 227 170 182 125 155
 140 197 162   5 231 229 126 106 243 224  90 206 209 214]
SET 3:  [212 147 175 188 221 217  48 222 240 189 198 192 148 122 232  50  56 195
  17  89  33   2 184 110  32 169  63  66  31  86 153 163 137 127 165 100
  43 142 118  21 176 159  44  37  40   0  16  19  71 205]
SET 4:  [174 141  85 109  47 238  24   4 201  70  60  38  93 178 124 226  69   1
 146   6  39 166 

In [20]:

# Authors file generation

print("---------- AUTHORS GENERATION STARTED ----------")

with open(authors_file_path, mode='w', newline='') as authors_file:
    authors_writer = csv.writer(authors_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    authors_writer.writerow(["Author", "Paper"])
    for index, author in enumerate(authors):
        percentage = 100*index/authors_number
        if percentage % 10 == 0:
            print(f"{int(index)}/{authors_number} ({int(percentage)}/100%)")
        # An author writes a number of paper between 1 and paper_fraction
        author_papers_number = rn.randint(1, (papers_number-1))
        papers_written = np.random.choice(papers, author_papers_number).tolist()
        papers_written = set(papers_written)
        if len(papers_written) > 1:
            papers_written = map(str, list(papers_written))
            papers_written = ";".join(papers_written)
        authors_writer.writerow([author, papers_written])
    print(f"{authors_number}/{authors_number} (100/100%)")
authors_file.close()
        
print("---------- AUTHORS GENERATION ENDED ----------")


---------- AUTHORS GENERATION STARTED ----------
0/25 (0/100%)
5/25 (20/100%)
10/25 (40/100%)
15/25 (60/100%)
20/25 (80/100%)
25/25 (100/100%)
---------- AUTHORS GENERATION ENDED ----------


In [21]:

# Info file generation

print("---------- INFO GENERATION STARTED ----------")

info_dataframe = pd.DataFrame(columns=["Dataset", "Paper", "Reader", "Rating", "Author"])
info_dataframe = info_dataframe.append(
    {
        "Dataset": dataset_name, 
        "Paper": papers_number, 
        "Reader": readers_number, 
        "Rating": ratings_number, 
        "Author": authors_number
    }, ignore_index=True)
info_dataframe.to_csv(info_file_path, index=False)

print("---------- INFO GENERATION ENDED ----------")

---------- INFO GENERATION STARTED ----------
---------- INFO GENERATION ENDED ----------


True
